In [ ]:
!git clone https://github.com/mueedurrehman/distiller.git
%cd distiller
!pip3 install -e .

fatal: destination path 'distiller' already exists and is not an empty directory.
/content/distiller
Obtaining file:///content/distiller
  Found existing installation: distiller 0.4.0rc0
    Can't uninstall 'distiller'. No files were found to uninstall.
  Running setup.py develop for distiller


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
import torchvision.models as models
from torchsummary import summary
import matplotlib.pyplot as plt
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import time
import math
import copy
import torchnet as tnt
from ipywidgets import widgets, interact
from copy import deepcopy
from collections import OrderedDict


from collections import OrderedDict
import os
import pandas as pd
import distiller.models
import distiller.quantization as quant
import distiller

In [ ]:
#helper functions

transformTest = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))])

test = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform = transformTest)
testLoader = torch.utils.data.DataLoader(test, batch_size=256,
                                         shuffle=False, num_workers=2)
                                         
baseModelCheckpoint = torch.load("/content/drive/MyDrive/CS6787 - Final Project/Google Colab Scripts/Baseline Models/baseResnet56")

baseModel = distiller.models.create_model(False, "cifar10", "resnet56_cifar", True, None)
baseModel.load_state_dict(baseModelCheckpoint["state_dict"])

# compute parameters for base model
# TODO: USE DISTILLER FOR COMPUTING SPARSITY. distiller.utils.model_sparsity(model)
# the below also gives correct counts
# def computeTotalParameters(model): 
#   return sum(p.numel() for p in model.parameters() if p.dim() in [2,4])

def to_sparse(x):
    """ converts dense tensor x to sparse format """
    x_typename = torch.typename(x).split('.')[-1]
    sparse_tensortype = getattr(torch.sparse, x_typename)

    indices = torch.nonzero(x)
    if len(indices.shape) == 0:  # if all elements are zeros
        return sparse_tensortype(*x.shape)
    indices = indices.t()
    values = x[tuple(indices[i] for i in range(indices.shape[0]))]
    return sparse_tensortype(indices, values, x.size())

def computeStateSizes(state_dict, compressed = True):
  originalSize = 0
  compressedSize = 0
  torch.save(state_dict, "./curStateDict")
  totalSize = os.path.getsize("./curStateDict")
  if compressed:
    for value in state_dict.values():
      torch.save(value, "denseTensor")
      sparse = to_sparse(value)
      torch.save(sparse.indices(), "sparseTensorIndices")
      torch.save(sparse.values(), "sparseTensorValues")
      torch.save(sparse.size(), "sparseTensorSize")
      originalSize += os.path.getsize("./denseTensor")
      compressedSize += os.path.getsize("./sparseTensorIndices") + os.path.getsize("./sparseTensorValues") + os.path.getsize("./sparseTensorSize")
    os.remove("./denseTensor")
    os.remove("./sparseTensorIndices")
    os.remove("./sparseTensorValues")
    os.remove("./sparseTensorSize")
    return (totalSize - originalSize + compressedSize) / (1024*1024)
  else:
    return totalSize

# TODO: Use this for computing sparsity instead
# def computeTotalParameters(model):
#   return sum(p.numel() for p in model.parameters() if p.requires_grad)

# def computeZeroParameters(model):
#   return sum(p.numel() - p.nonzero().size(0) for p in model.parameters() if p.requires_grad)

# def computeSparsity(model):
#   return round(computeZeroParameters (model) / computeTotalParameters(model) * 100, 3)

def computeSize(model):
  torch.save(model, "./curModel")
  size = os.path.getsize("./curModel")
  os.remove("./curModel")
  return size / (1024*1024)

def computeCompressedModelSize(model):
  stateSize = computeStateSizes(model.state_dict(), True)
  return stateSize + computeSize(model) - computeStateSizes(model.state_dict(), False)

def computeTotalParameters(model):
  return sum(p.nonzero().size(0) for p in model.parameters())

def computeTotalTrainableParameters(model):
  return sum(p.nonzero().size(0) for p in model.parameters() if p.requires_grad)

baseParameters = computeTotalParameters(baseModel)
baseTrainParameters = computeTotalTrainableParameters(baseModel)
baseSize = computeSize(baseModel)

def computeIndexSavingRate(baseParameters, compressedParameters):
  return (baseParameters - compressedParameters)/compressedParameters

def computeStatistics(compressedModel):
  modelParameters = computeTotalParameters(compressedModel)
  modelTrainParameters = computeTotalTrainableParameters(compressedModel)d
  modelSize = computeCompressedModelSize(compressedModel)
  indexSavingRate = computeIndexSavingRate(baseParameters, modelParameters)
  indexSavingRateTrain = computeIndexSavingRate(baseTrainParameters, modelTrainParameters)
  parameterCompressionRate = baseParameters / modelparameters
  parameterCompressionRateTrain =  baseTrainParameters / modelTrainParameters
  size = computeSize(compressedModel)
  sizeCompressionRate = baseSize / size
  return {"Size Compression Rate" : sizeCompressionRate, "ParameterCompressionRate" : parameterCompressionRate, "IndexSavingRate": indexSavingRate, 
          "Train Parameter Compression Rate" : parameterCompressionRateTrain, "Train Index Saving Rate" : indexSavingRateTrain}

def test(net):
    net.eval()
    test_loss = 0
    correct = 0
    total = 0
    criterion = nn.CrossEntropyLoss()
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(testLoader):
            inputs, targets = inputs.to('cuda'), targets.to('cuda')
            outputs = net(inputs)
            loss = criterion(outputs, targets)
            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()
    acc = 100.*correct/total
    return acc

  0%|          | 8192/170498071 [00:00<36:47, 77225.04it/s]

170500096it [00:01, 97140725.72it/s]                               


Extracting ./data/cifar-10-python.tar.gz to ./data


In [ ]:
test(baseModel)

92.94

In [ ]:
def removeModule(checkpoint):
  state_dictionary = checkpoint["state_dict"]
  newDict = {}
  for key,value in state_dictionary.items():
    newKey = key[7:]
    newDict[key] = value
  checkpoint["state_dict"] = newDict
  return checkpoint

# Seeing Layers

In [ ]:
# Structure Pruning
testere = torch.load("/content/drive/MyDrive/CS6787 - Final Project/Google Colab Scripts/Pruning Models/Structure Pruning/ActivationAPoZRankedFilterPruner/Averagepercentage of zeros (APoZ) V1/best.pth.tar")
print(len(testere["state_dict"].keys()))
testere["state_dict"].keys()

344


odict_keys(['module.conv1.weight', 'module.bn1.weight', 'module.bn1.bias', 'module.bn1.running_mean', 'module.bn1.running_var', 'module.bn1.num_batches_tracked', 'module.layer1.0.conv1.weight', 'module.layer1.0.bn1.weight', 'module.layer1.0.bn1.bias', 'module.layer1.0.bn1.running_mean', 'module.layer1.0.bn1.running_var', 'module.layer1.0.bn1.num_batches_tracked', 'module.layer1.0.conv2.weight', 'module.layer1.0.bn2.weight', 'module.layer1.0.bn2.bias', 'module.layer1.0.bn2.running_mean', 'module.layer1.0.bn2.running_var', 'module.layer1.0.bn2.num_batches_tracked', 'module.layer1.1.conv1.weight', 'module.layer1.1.bn1.weight', 'module.layer1.1.bn1.bias', 'module.layer1.1.bn1.running_mean', 'module.layer1.1.bn1.running_var', 'module.layer1.1.bn1.num_batches_tracked', 'module.layer1.1.conv2.weight', 'module.layer1.1.bn2.weight', 'module.layer1.1.bn2.bias', 'module.layer1.1.bn2.running_mean', 'module.layer1.1.bn2.running_var', 'module.layer1.1.bn2.num_batches_tracked', 'module.layer1.2.conv1

In [ ]:
# Base Model
print(len(baseModelCheckpoint["state_dict"].keys()))
baseModelCheckpoint["state_dict"].keys()

344


dict_keys(['module.conv1.weight', 'module.bn1.weight', 'module.bn1.bias', 'module.bn1.running_mean', 'module.bn1.running_var', 'module.bn1.num_batches_tracked', 'module.layer1.0.conv1.weight', 'module.layer1.0.bn1.weight', 'module.layer1.0.bn1.bias', 'module.layer1.0.bn1.running_mean', 'module.layer1.0.bn1.running_var', 'module.layer1.0.bn1.num_batches_tracked', 'module.layer1.0.conv2.weight', 'module.layer1.0.bn2.weight', 'module.layer1.0.bn2.bias', 'module.layer1.0.bn2.running_mean', 'module.layer1.0.bn2.running_var', 'module.layer1.0.bn2.num_batches_tracked', 'module.layer1.1.conv1.weight', 'module.layer1.1.bn1.weight', 'module.layer1.1.bn1.bias', 'module.layer1.1.bn1.running_mean', 'module.layer1.1.bn1.running_var', 'module.layer1.1.bn1.num_batches_tracked', 'module.layer1.1.conv2.weight', 'module.layer1.1.bn2.weight', 'module.layer1.1.bn2.bias', 'module.layer1.1.bn2.running_mean', 'module.layer1.1.bn2.running_var', 'module.layer1.1.bn2.num_batches_tracked', 'module.layer1.2.conv1.

In [ ]:
# Resnet56_cifat10_post_train+retraining_10bit
testere = torch.load("/content/drive/MyDrive/CS6787 - Final Project/Google Colab Scripts/Quantization Models/Resnet56_cifat10_post_train+retraining_10bit/esterov=True_best.pth.tar")
print(len(testere["state_dict"].keys()))
testere["state_dict"].keys()

969


odict_keys(['module.conv1.float_weight', 'module.conv1.weight', 'module.conv1.weight_scale', 'module.conv1.weight_zero_point', 'module.bn1.weight', 'module.bn1.bias', 'module.bn1.running_mean', 'module.bn1.running_var', 'module.bn1.num_batches_tracked', 'module.relu.fake_q.ema_decay', 'module.relu.fake_q.tracked_min_biased', 'module.relu.fake_q.tracked_min', 'module.relu.fake_q.tracked_max_biased', 'module.relu.fake_q.tracked_max', 'module.relu.fake_q.iter_count', 'module.relu.fake_q.scale', 'module.relu.fake_q.zero_point', 'module.layer1.0.conv1.float_weight', 'module.layer1.0.conv1.weight', 'module.layer1.0.conv1.weight_scale', 'module.layer1.0.conv1.weight_zero_point', 'module.layer1.0.bn1.weight', 'module.layer1.0.bn1.bias', 'module.layer1.0.bn1.running_mean', 'module.layer1.0.bn1.running_var', 'module.layer1.0.bn1.num_batches_tracked', 'module.layer1.0.relu1.fake_q.ema_decay', 'module.layer1.0.relu1.fake_q.tracked_min_biased', 'module.layer1.0.relu1.fake_q.tracked_min', 'module.la

In [ ]:
# Resnet56_cifar10_post_train_2
entry = torch.load(pather)
print(len(entry["state_dict"].keys()))
entry["state_dict"].keys()

858


odict_keys(['module.conv1.num_forwards', 'module.conv1.force_readjust', 'module.conv1.output_scale', 'module.conv1.output_zero_point', 'module.conv1.w_scale', 'module.conv1.w_zero_point', 'module.conv1.fp_bias', 'module.conv1.accum_scale', 'module.conv1.is_simulated_quant_weight_shifted', 'module.conv1.wrapped_module.weight', 'module.conv1.wrapped_module.bias', 'module.layer1.0.conv1.num_forwards', 'module.layer1.0.conv1.force_readjust', 'module.layer1.0.conv1.output_scale', 'module.layer1.0.conv1.output_zero_point', 'module.layer1.0.conv1.w_scale', 'module.layer1.0.conv1.w_zero_point', 'module.layer1.0.conv1.fp_bias', 'module.layer1.0.conv1.accum_scale', 'module.layer1.0.conv1.is_simulated_quant_weight_shifted', 'module.layer1.0.conv1.wrapped_module.weight', 'module.layer1.0.conv1.wrapped_module.bias', 'module.layer1.0.conv2.num_forwards', 'module.layer1.0.conv2.force_readjust', 'module.layer1.0.conv2.output_scale', 'module.layer1.0.conv2.output_zero_point', 'module.layer1.0.conv2.w_s

In [ ]:
testentry = removeModule(torch.load(pather))

In [ ]:
entry["state_dict"]

OrderedDict([('module.conv1.num_forwards', tensor([40], device='cuda:0')),
             ('module.conv1.force_readjust', tensor(False, device='cuda:0')),
             ('module.conv1.output_scale', tensor([11.0280], device='cuda:0')),
             ('module.conv1.output_zero_point', tensor([0.], device='cuda:0')),
             ('module.conv1.w_scale', tensor([[[[1.0050e+02]]],
              
              
                      [[[2.1752e+06]]],
              
              
                      [[[6.9460e+05]]],
              
              
                      [[[1.1469e+02]]],
              
              
                      [[[8.4651e+01]]],
              
              
                      [[[5.9710e+05]]],
              
              
                      [[[4.1348e+01]]],
              
              
                      [[[1.2563e+06]]],
              
              
                      [[[6.6465e+01]]],
              
              
                      [[[3.9228e+

In [ ]:
testentry = removeModule(torch.load(pather))

In [ ]:
# https://github.com/IntelLabs/distiller/blob/master/distiller/quantization/quantizer.py

In [ ]:
## Post-Train Quantize with Distiller
model = distiller.models.create_model(False, "cifar10", "resnet56_cifar", True, None)
# entry = removeModule(torch.load(pather))
# entry = torch.load(pather)
# Load the file

quant_mode = {'activations': 'ASYMMETRIC_UNSIGNED', 'weights': 'SYMMETRIC'}
stats_file = "/content/drive/MyDrive/CS6787 - Final Project/Google Colab Scripts/Quantization Models/Resnet56 Quantization Stats/resnet56_quant_stats.yaml"
dummy_input = distiller.get_dummy_input(input_shape=model.input_shape)

quantizer = quant.PostTrainLinearQuantizer(
    deepcopy(model), bits_activations=8, bits_parameters=8, mode=quant_mode,
    model_activation_stats=stats_file, overrides=None

quantizer.prepare_model(dummy_input)

SyntaxError: ignored

# "Prepare_Model

In [ ]:
pather = "/content/drive/MyDrive/CS6787 - Final Project/Google Colab Scripts/Quantization Models/Resnet56_cifar10_post_train_2_16bit/quantized_checkpoint.pth.tar"

In [ ]:
model = distiller.models.create_model(False, "cifar10", "resnet56_cifar", True, None)
quant_mode = {'activations': 'ASYMMETRIC_UNSIGNED', 'weights': 'ASYMMETRIC_UNSIGNED'}
stats_file = "/content/drive/MyDrive/CS6787 - Final Project/Google Colab Scripts/Quantization Models/Resnet56 Quantization Stats/resnet56_quant_stats.yaml"
dummy_input = distiller.get_dummy_input(input_shape=model.input_shape)

quantizer = quant.PostTrainLinearQuantizer(
    deepcopy(model), bits_activations=16, bits_parameters=16, mode=quant_mode,
    model_activation_stats=stats_file, overrides=None
)
quantizer.prepare_model(dummy_input)

In [ ]:
print(type(quantizer.model))
quantizer.model

<class 'torch.nn.parallel.data_parallel.DataParallel'>


In [ ]:
type(model)
Checkpoint = torch.load(pather)
Checkpoint

{'arch': 'resnet56_cifar',
 'compression_sched': {'masks_dict': {'module.bn1.bias': None,
   'module.bn1.weight': None,
   'module.conv1.weight': None,
   'module.fc.bias': None,
   'module.fc.weight': None,
   'module.layer1.0.bn1.bias': None,
   'module.layer1.0.bn1.weight': None,
   'module.layer1.0.bn2.bias': None,
   'module.layer1.0.bn2.weight': None,
   'module.layer1.0.conv1.weight': None,
   'module.layer1.0.conv2.weight': None,
   'module.layer1.1.bn1.bias': None,
   'module.layer1.1.bn1.weight': None,
   'module.layer1.1.bn2.bias': None,
   'module.layer1.1.bn2.weight': None,
   'module.layer1.1.conv1.weight': None,
   'module.layer1.1.conv2.weight': None,
   'module.layer1.2.bn1.bias': None,
   'module.layer1.2.bn1.weight': None,
   'module.layer1.2.bn2.bias': None,
   'module.layer1.2.bn2.weight': None,
   'module.layer1.2.conv1.weight': None,
   'module.layer1.2.conv2.weight': None,
   'module.layer1.3.bn1.bias': None,
   'module.layer1.3.bn1.weight': None,
   'module.lay

In [ ]:
quantizer.model.load_state_dict(Checkpoint["state_dict"])

RuntimeError: ignored

In [ ]:
# Here we trigger the conversion via the Quantizer instance. Later on we show another way which does not
# require the quantizer
pyt_model = quantizer.convert_to_pytorch(dummy_input)

# Note that the converted model is automatically moved to the CPU, regardless
# of the device of the Distiller model
print('Distiller model device:', distiller.model_device(quantizer.model))
print('PyTorch model device:', distiller.model_device(pyt_model))

AssertionError: ignored

In [ ]:
test(pyt_model)

RuntimeError: ignored

# API Conversion

In [ ]:
# Quantization
# pather = "/content/drive/MyDrive/CS6787 - Final Project/Google Colab Scripts/Quantization Models/Resnet56_cifar10_post_train_2_8bit/quantized_checkpoint.pth.tar"
# pather = "/content/drive/MyDrive/CS6787 - Final Project/Google Colab Scripts/Quantization Models/Resnet56_cifar10_post_train_2_4bit/quantized_checkpoint.pth.tar"
# pather = "/content/drive/MyDrive/CS6787 - Final Project/Google Colab Scripts/Quantization Models/Resnet56_cifar10_post_train_2/quantized_checkpoint.pth.tar"
# pather = "/content/drive/MyDrive/CS6787 - Final Project/Google Colab Scripts/Quantization Models/Resnet56_cifar10_post_train_2_10bit/quantized_checkpoint.pth.tar"
# pather = "/content/drive/MyDrive/CS6787 - Final Project/Google Colab Scripts/Quantization Models/Resnet56_cifar10_post_train_2_12bit/quantized_checkpoint.pth.tar"
pather = "/content/drive/MyDrive/CS6787 - Final Project/Google Colab Scripts/Quantization Models/Resnet56_cifar10_post_train_2_16bit/quantized_checkpoint.pth.tar"

In [ ]:
# Quantization + Retraining
# pather = "/content/drive/MyDrive/CS6787 - Final Project/Google Colab Scripts/Quantization Models/Resnet56_cifat10_post_train+retraining_8bit/esterov=True_best.pth.tar"
# pather = "/content/drive/MyDrive/CS6787 - Final Project/Google Colab Scripts/Quantization Models/Resnet56_cifat10_post_train+retraining_10bit/esterov=True_best.pth.tar"

In [ ]:
# Load Checkpoint
model = distiller.models.create_model(False, "cifar10", "resnet56_cifar", True, None)
model = distiller.apputils.load_lean_checkpoint(model, pather)
dummy_input = distiller.get_dummy_input(input_shape=model.input_shape)

In [ ]:
# # Load Checkpoint
# model = distiller.models.create_model(False, "cifar10", "resnet56_cifar", True, None)
# dummy_input = distiller.get_dummy_input(input_shape=model.input_shape)
# # quantizer = quant.PostTrainLinearQuantizer.from_args(model, args)
# model.prepare_model(dummy_input)
# # model = distiller.apputils.load_lean_checkpoint(model, pather)
# # dummy_input = distiller.get_dummy_input(input_shape=model.input_shape)

AttributeError: ignored

In [ ]:
# Convert and Evaluate
model_pyt_model = distiller.quantization.convert_distiller_ptq_model_to_pytorch(model, dummy_input)
# Run evaluation
test(model_pyt_model)

AssertionError: ignored

In [ ]:
computeSize(model_pyt_model)

RuntimeError: ignored

# Ignore

In [ ]:
model = distiller.models.create_model(False, "cifar10", "resnet56_cifar", True, None)
# entry = removeModule(torch.load("/content/drive/MyDrive/Colab Notebooks/6787 Notebooks/Google Colab Scripts/baseOneShot/best.pth.tar")
# entry = removeModule(torch.load(pather))
# model = distiller.models.create_model(False, "cifar10", "resnet56_cifar", True, None)
# Load the file
# model.load_state_dict(entry["state_dict"])
print(computeStateSizes(model.state_dict(), True))
print(computeStateSizes(model.state_dict(), False))
entry = torch.load("../drive/MyDrive/CS6787 - Final Project/Google Colab Scripts/Pruning Models/Element Pruning/Direct Pruning/best.pth.tar", map_location=torch.device('cpu'))
startTime = time.time()
finalAcc = test(model)
finalTime = time.time() - startTime
stats = computeStatistics(model)
# stats["Name"] = entry.name
# stats["Time"] = finalTime
# results.append(stats)

# df = pd.DataFrame(results)

stats

RuntimeError: ignored

In [ ]:
print(baseParameters, baseTrainParameters, baseSize)

855770 855770 3.41556453704834


In [ ]:
modelDirectory = "/content/drive/MyDrive/CS6787 - Final Project/Google Colab Scripts/Quantization Models/Post-Training Quantization 02/quantized_checkpoint.pth.tar"

results = []
# for entry in os.scandir(modelDirectory):
model = distiller.models.create_model(False, "cifar10", "resnet56_cifar", True, None)
#load the file
torch.load(modelDirectory)
model.load_state_dict(entry["state_dict"])
startTime = time.time()
finalAcc = test(model)
finalTime = time.time() - startTime
stats = computeStatistics(model)
stats["Name"] = entry.name
stats["Time"] = finalTime
results.append(stats)

df = pd.DataFrame(results)

save_dir = "."
df.to_csv(save_dir)

TypeError: ignored